In [3]:
#criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import pathlib
import time

driver = webdriver.Chrome()

produtos_df = pd.read_excel("buscas.xlsx")
#display(produtos_df)

def busca_google_shopping(driver, produto, termos_ban, preco_min, preco_max):
    driver.get("https://www.google.com/")
    #pesquisar produto
    produto = produto.lower()
    termos_ban = termos_ban.lower()
    lista_termos_ban = termos_ban.split(" ")
    lista_termos_prod = produto.split(" ")
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    
    #pesquisando no google
    driver.find_element('xpath', "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(produto)
    driver.find_element('xpath', "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(Keys.ENTER)
    #entrar na aba shopping
    elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if 'Shopping' in item.text:
            item.click()
            break
    time.sleep(5)
    #pegar as informações do produto
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'KZmu8e')
    
    lista_ofertas = []
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'sh-np__product-title').text
        nome = nome.lower()
        if produto in nome:
            tem_termos_ban = False
            for termo in lista_termos_ban:
                if termo in nome:
                    tem_termos_ban = True

            tem_todos_termos_prod = True
            for termo in lista_termos_prod:
                if termo not in nome:
                    tem_todos_termos_prod = False

            if not tem_termos_ban and tem_todos_termos_prod:
                preco = resultado.find_element(By.CLASS_NAME, 'T14wmb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = preco[0:7]
                preco = float(preco)

                if preco_min <= preco <= preco_max:
                    elemento_ref = resultado.find_element(By.CLASS_NAME, 'ROMz4c')
                    elemento_pai = elemento_ref.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    #print(nome, preco, link)
                    lista_ofertas.append((nome, preco, link))
    return lista_ofertas
  
def busca_buscape(driver, produto, termos_ban, preco_min, preco_max):
    #tratando valores
    produto = produto.lower()
    termos_ban = termos_ban.lower()
    lista_termos_ban = termos_ban.split(" ")
    lista_termos_prod = produto.split(" ")
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    
    #buscar no buscapé
    driver.get("https://www.buscape.com.br/")
    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    time.sleep(5)
    
    #pegar resultados
    lista_resultados = driver.find_elements(By.CLASS_NAME, "SearchCard_ProductCard_Inner__7JhKb")
    lista_ofertas = []
    
    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
        nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
        if produto in nome:
            link = resultado.get_attribute('href')
            #print(nome, preco, link)

            #analisar resultados e os termos
            tem_termos_ban = False
            for termo in lista_termos_ban:
                if termo in nome:
                    tem_termos_ban = True

            tem_todos_termos_prod = True
            for termo in lista_termos_prod:
                if termo not in nome:
                    tem_todos_termos_prod = False

            #analisar preço está entre preço min e preço max
            if not tem_termos_ban and tem_todos_termos_prod:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = preco[0:7]
                preco = float(preco)
                if preco_min <= preco <= preco_max:
                    lista_ofertas.append((nome, preco, link))
    return lista_ofertas

In [4]:
#definindo as variaveis e chamando as funções
tabela_ofertas = pd.DataFrame()
for linha in produtos_df.index:
    produto = produtos_df.loc[linha, "Nome"]
    termos_ban = produtos_df.loc[linha, "Termos banidos"]
    preco_min = produtos_df.loc[linha, "Preço mínimo"]
    preco_max = produtos_df.loc[linha, "Preço máximo"]


    lista_ofertas_google_shopping = busca_google_shopping(driver, produto, termos_ban, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
    lista_ofertas_buscape = busca_buscape(driver, produto, termos_ban, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None
display(tabela_ofertas)

C:\Users\gabriel\AppData\Local\Temp\ipykernel_1564\1443282452.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\gabriel\AppData\Local\Temp\ipykernel_1564\1443282452.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\gabriel\AppData\Local\Temp\ipykernel_1564\1443282452.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_buscape)


,produto,preco,link
0,vitrine - apple iphone 12 64gb - original + ga...,3324.05,https://www.google.com/aclk?sa=l&ai=DChcSEwj0m...
1,iphone 12 64gb - azul - sou como novo,3059.10,https://www.google.com/aclk?sa=l&ai=DChcSEwj0m...
2,iphone 12 64gb - preto - sou como novo,3059.10,https://www.google.com/aclk?sa=l&ai=DChcSEwj0m...
3,iphone 12 64gb - branco - sou como novo,3059.10,https://www.google.com/aclk?sa=l&ai=DChcSEwj0m...
4,iphone 12 64gb - roxo - sou como novo,3059.10,https://www.google.com/aclk?sa=l&ai=DChcSEwj0m...
5,iphone 12 64gb - verde - sou como novo,3059.10,https://www.google.com/aclk?sa=l&ai=DChcSEwj0m...
6,iphone 12 64gb - (product)red - sou como novo,3059.10,https://www.google.com/aclk?sa=l&ai=DChcSEwj0m...
0,workstation intermédia a intel core i5 10400f ...,4468.50,https://www.google.com/aclk?sa=l&ai=DChcSEwjio...
0,placa de video nvidia geforce rtx 3060 12 gb g...,4441.10,https://www.buscape.com.br/placa-de-video/plac...


In [5]:
#exportando para excel
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

In [6]:
#enviar email
fromaddr = # De...
toaddr = # Para...
msg = MIMEMultipart()

msg['From'] = fromaddr
msg['To'] = toaddr
msg['Subject'] = 'Produto(s) Encontrado(s) na faixa de preço desejada'

body = f"""
<p>Prezado(s),</p>
<p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada</p>
{tabela_ofertas.to_html(index=False)}
<p>Att.,</p>
"""

#Aqui começa a parte do Anexo
msg.attach(MIMEText(body, 'html'))
filename = ('Ofertas.xlsx')
attachment = open(filename,'rb')
part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', f'attachment; filename={filename}')
msg.attach(part)
attachment.close()


s = smtplib.SMTP('smtp.gmail.com: 587')
s.starttls()
s.login(fromaddr, #Senha do app )
text = msg.as_string()
s.sendmail(fromaddr, toaddr, msg.as_string().encode('utf-8'))
s.quit()

print('E-mail enviado')

E-mail enviado
